## IMPORTS

In [ ]:
import os # Manipular arquivos
import pandas as pd # Manipulação de dados
import pyodbc # Conectar ao banco de dados
from datetime import datetime # Data 
from openpyxl import Workbook, load_workbook # Editar Excel

# Desabilitar warnings
import warnings
warnings.filterwarnings("ignore")

## QUERIES

### Removi as queries por questões de segurança

In [ ]:
querys = {
# Query para clientes ativos
'vidas' : '''
SELECT * FROM DADOS''',


# Query para consultas
'consultas' : '''
SELECT * FROM DADOS''',


# Query para questões
'questoes' : '''
SELECT * FROM DADOS''',


# Query para riscos
'riscos' : '''
SELECT * FROM DADOS''',

'plano' : '''
SELECT * FROM DADOS''',

# QUERY para os ultimos registros de imc
'imc' : '''
SELECT * FROM DADOS''',
 

# Query para CID
'cid' : '''
SELECT * FROM DADOS'''}

## Funções

In [ ]:
def trocar_valores_quest(coluna):
    dict = {
        1: "Pressão arterial (mmHg)",
        2: "Glicemia pós prandial (mg/dl)",
        3: "Peso (Kg)",
        4: "Altura (m)",
        5: "IMC",
        6: "Glicemia ao acaso (mg/dl)",
        7: "Colesterol total (mg/dl)",
        8: "Colesterol HDL (mg/dl)",
        9: "Triglicerídeos (mg/dl)",
        10: "Fuma?",
        11: "Você é diabético?",
        12: "Grau de dependência de Atividades Diárias - AVD's",
        13: "Ansiedade",
        14: "Asma/Bronquite",
        15: "Hipertensão arterial",
        653: "Idade Gestacional (Em semanas)",
        2269: "DPP (Data Prevista do Parto)"
    }
    return dict.get(coluna, "Outros")

def trocar_valores_risco(coluna):
    dict = {
        1 : "Risco Cardiovascular",
        2 : "Risco Emocional",
        3 : "Risco Complementar Masculino",
        7 : "Risco Clínico Funcional",
        8 : "Risco Complexo",
        9 : "Risco Global",
        10 : "Risco Saúde Mental",
        11 : "Risco de Hospitalização",
        12 : "Risco Complementar Feminino",
        13 : "Risco Diabetes",
        16: "Risco Doença Pulmonar Obstrutiva Crônica",
        17: "Risco ITB Tibial - Direita",
        18: "Risco ITB Tibial - Esquerda",
        19: "Risco ITB Pediosa - Direita",
        20: "Risco ITB Pediosa - Esquerda",
        21: "Risco Sintomas Neuropáticos (ESN)",
        22: "Risco Comprometimento Neuropático",
        23: "Risco Perda da Sensibilidade Protetora dos Pés",
        24: "Risco IVCF-20"}
    return dict.get(coluna, "Outros")



def categorizar_doenca(codigo):
    if codigo >= 'A00' and codigo <= 'B99':
        return 'Infecciosas e parasitárias'
    elif codigo >= 'C00' and codigo <= 'D48':
        return 'Tumores'
    elif codigo >= 'D50' and codigo <= 'D89':
        return 'Sangue e dos órgãos hematopoéticos'
    elif codigo >= 'E00' and codigo <= 'E90':
        return 'Endócrinas, nutricionais e metabólicas'
    elif codigo >= 'F00' and codigo <= 'F99':
        return 'Transtornos mentais e comportamentais'
    elif codigo >= 'G00' and codigo <= 'G99':
        return 'Sistema nervoso'
    elif codigo >= 'H00' and codigo <= 'H59':
        return 'Olho'
    elif codigo >= 'H60' and codigo <= 'H95':
        return 'Ouvido'
    elif codigo >= 'I00' and codigo <= 'I99':
        return 'Aparelho circulatório'
    elif codigo >= 'J00' and codigo <= 'J99':
        return 'Aparelho respiratório'
    elif codigo >= 'K00' and codigo <= 'K93':
        return 'Aparelho digestivo'
    elif codigo >= 'L00' and codigo <= 'L99':
        return 'Pele'
    elif codigo >= 'M00' and codigo <= 'M99':
        return 'Sistema osteomuscular'
    elif codigo >= 'N00' and codigo <= 'N99':
        return 'Aparelho geniturinário'
    elif codigo >= 'O00' and codigo <= 'O99':
        return 'Gravidez e parto'
    elif codigo >= 'P00' and codigo <= 'P96':
        return 'Afecções perinatais'
    elif codigo >= 'Q00' and codigo <= 'Q99':
        return 'Malformações congênitas'
    elif codigo >= 'R00' and codigo <= 'R99':
        return 'Sintomas e sinais'
    elif codigo >= 'S00' and codigo <= 'T98':
        return 'Lesões e envenenamentos'
    elif codigo >= 'V01' and codigo <= 'Y98':
        return 'Causas externas'
    elif codigo >= 'Z00' and codigo <= 'Z99':
        return 'Fatores de influência na saúde'
    elif codigo >= 'U04' and codigo <= 'U99':
        return 'Códigos especiais da OMS'
    else:
        return 'Código não pertence a nenhum capítulo conhecido do CID-10'


# Funções para ajustar os valores de idade

def calcular_idade(df):
    def idade(idade):
        if idade > 100 or idade < 0 or pd.isnull(idade):
            return -1
        else:
            return idade

    data_atual = pd.to_datetime(datetime.now().date())
    df['DT_PSSOA_NASCI'] = pd.to_datetime(df['DT_NASCIMENTO'], errors='coerce')
    df['idade'] = (data_atual - df['DT_NASCIMENTO']).dt.days // 365
    df['idade'] = df['idade'].apply(lambda x: idade(x))

## Carregar Servidores

In [ ]:
# Carregar informações dos servidores e nomes formatados

with open('path/servidores.txt', 'r') as arquivo:
	servidores = arquivo.read()

with open('path/nomes_formatados.txt', 'r') as arquivo:
    linhas = arquivo.readlines()
    nomes_formatados = {}
    for linha in linhas:
        chave, valor = linha.strip().split(':')
        nomes_formatados[chave.strip()] = valor.strip()

#### Função para salvar dados de varias querys em varios databases

In [ ]:
# Função para acessar o servidor, executar as querys e armazenar os resultados
def acessar_servidor_salvar_dados(querys):
    arquivo_excel = 'vidas.xlsx'
    wb = load_workbook(arquivo_excel)
    ws = wb.create_sheet("vidas") if "diario" not in wb.sheetnames else wb["diario"]
    if ws.max_row == 0:
        ws.append(['Cliente', 'Vidas', 'Data'])

    df_dict = {}
    data_atual = datetime.now()

    # Limpeza dos Dados
    for servidor in servidores.split('\n\n'):
        t = servidor.split('\n')[2].replace(';databaseName', '').replace('jdbc:sqlserver://', '').split('=')
        url = t[1]
        databaseName = t[2]
        username = servidor.split('\n')[3].split('=')[1]
        password = servidor.split('\n')[4].split('=')[1]
        drive = 'SQL Server'
        link = f'DRIVER={drive};SERVER={url};DATABASE={databaseName};UID={username};PWD={password}'
        try:
            # Estabelecendo a conexão
            conn = pyodbc.connect(link)
            if conn:
                cursor = conn.cursor()
    
                cursor.execute(next(iter(querys.values())))
                rows = cursor.fetchall()
                empresas = nomes_formatados[databaseName]
                vidas = len(rows)
                ws.append([empresas, vidas, data_atual])
                
            for query_name, query in querys.items():
                cursor.execute(query)
                rows = cursor.fetchall()
                columns = [column[0] for column in cursor.description]
                df_temp = pd.DataFrame.from_records(rows, columns=columns)
                df_temp['empresa'] = empresas
                
                # Adiciona os resultados ao dicionário usando o nome da query como chave
                if query_name not in df_dict:
                    df_dict[query_name] = df_temp
                else:
                    df_dict[query_name] = pd.concat([df_dict[query_name], df_temp])
            
            cursor.close()
            conn.close()
        # Erro        
        except pyodbc.Error as e:
            print(nomes_formatados[databaseName])
            print("Ocorreu um erro ao conectar ao banco de dados:", e)
            print()

    # Salva as alterações no arquivo Excel
    wb.save(arquivo_excel)
    return df_dict



# Função para salvar os resultados em um arquivo Excel com abas separadas
def salvar_resultados_excel(df_dict, file_path):
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        for query_name, df in df_dict.items():
            df.to_excel(writer, sheet_name=query_name, index=False)



# Recria o arquivo
def limpar_arquivo_excel(file_path):
    try:
        # Remove o arquivo se ele existir
        if os.path.exists(file_path):
            os.remove(file_path)
        # Cria um novo arquivo Excel vazio
        wb = Workbook()
        wb.save(file_path)
    except Exception as e:
        print(f"Ocorreu um erro ao apagar e criar o arquivo {file_path}: {e}")

### Aplicando as funções

In [ ]:
# Limpando o arquivo Excel
limpar_arquivo_excel('path/dados.xlsx')

# Acessa os servidores, executa as querys e armazena os resultados
resultados = acessar_servidor_salvar_dados(querys)
print('Resultados foram salvos!')

# Tratamento dos dados
resultados['questoes']['questao'] = resultados['questoes']['CODE_QUESTAO'].apply(trocar_valores_quest) # Adiciona nome da questão
resultados['riscos']['risco'] = resultados['riscos']['CODE_RISCO'].apply(trocar_valores_quest) # Adiciona nome do risco
resultados['cid']['categoria_doenca'] = resultados['cid']['COD_CID'].apply(categorizar_doenca) # Adiciona categoria da doença

imc_dfs = [resultados['imc'], resultados['imc_recente'], resultados['imc_antigo']]
[df.update({'IMC': df['IMC'].astype(str).str.replace('.', ',')}) for df in imc_dfs]

# Salvando os resultados em um arquivo Excel
salvar_resultados_excel(resultados, 'path/dados.xlsx')
print('Arquivo salvo com sucesso!')